In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

In [2]:
# RDD만을 이용한 데이터 추출
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [3]:
# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기
movie_att = movies_rdd.join(attendances_rdd)
movie_att.take(5)

[(2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [4]:
movie_att.filter(lambda x : x[1][0][1] == '마블' and x[1][1][0] >= 5000000).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [5]:
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [6]:
sc.stop()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [8]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [9]:
movie_schema = ["id", "name", "company", "year", "month", "day"]

In [10]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [11]:
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [12]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [13]:
df.show(3)

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
+---+--------+-------+----+-----+---+
only showing top 3 rows



In [14]:
df.select('company').show()

+-------+
|company|
+-------+
|   마블|
|     DC|
|     DC|
| 디즈니|
|   마블|
+-------+



In [15]:
df.filter(df.year > 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [16]:
df.select('year', 'month', 'day').show()

+----+-----+---+
|year|month|day|
+----+-----+---+
|2012|    4| 26|
|2013|    6| 13|
|2008|    8|  6|
|2014|    1| 16|
|2008|    4| 30|
+----+-----+---+



In [17]:
df.filter((df.company == '마블') | (df.company == 'DC')).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [18]:
df.createOrReplaceTempView("movies")

In [19]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [20]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT *
FROM movies
WHERE year > 2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [21]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT name, company
FROM movies
WHERE year < 2012
"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



In [22]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
SELECT *
FROM movies
WHERE name LIKE "%맨"
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [23]:
# BETWEEN 특정 데이터와 데이터 사이를 조회
# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8
query = """
SELECT *
FROM movies
WHERE month BETWEEN 4 AND 8
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [24]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [25]:
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [26]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [27]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [28]:
att_df.createOrReplaceTempView("att")

In [29]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [30]:
# df와 join
query = '''
SELECT movies.id, movies.name, movies.company, att.att
FROM movies
JOIN att ON movies.id = att.id
'''

spark.sql(query).show()

+---+--------+-------+-----------+
| id|    name|company|        att|
+---+--------+-------+-----------+
|  1|어벤져스|   마블|1.3934592E7|
|  2|  슈퍼맨|     DC|  2182227.0|
|  3|  배트맨|     DC|  4226242.0|
|  4|겨울왕국| 디즈니|1.0303058E7|
|  5|아이언맨|   마블|  4300365.0|
+---+--------+-------+-----------+



In [31]:
spark.stop()

In [32]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [35]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [36]:
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [37]:
data.createOrReplaceTempView("mobility_data")

In [38]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [39]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [40]:
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#354, 'count(1) AS trips#355]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#281 as string),  , -1)[0]], [split(cast(pickup_datetime#281 as string),  , -1)[0] AS pickup_date#354, count(1) AS trips#355L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#279,dispatching_base_num#280,pickup_datetime#281,dropoff_datetime#282,PULocationID#283,DOLocationID#284,SR_Flag#285])
      +- Relation [hvfhs_license_num#279,dispatching_base_num#280,pickup_datetime#281,dropoff_datetime#282,PULocationID#283,DOLocationID#284,SR_Flag#285] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#359], [_groupingexpression#359 AS pickup_date#354, count(1) AS trips#355L]
+- Project [split(cast(pickup_datetime#281 as string),  , -1)[0] AS _groupingexpression#359]
   +- Rel

In [41]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#363]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#362]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#362], [pickup_date#362, count(1) AS trips#363L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#281 as string),  , -1)[0] AS pickup_date#362]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#279,dispatching_base_num#280,pickup_datetime#281,dropoff_datetime#282,PULocationID#283,DOLocationID#284,SR_Flag#285])
            +- Relation [hvfhs_license_num#279,dispatching_base_num#280,pickup_datetime#281,dropoff_datetime#282,PULocationID#283,DOLocationID#284,SR_Flag#285] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#362], [pickup_date#362, count(1) AS tri

In [43]:
spark.stop()

In [44]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [45]:
# 운행 데이터프레임 생성, Zone 데이터프레임 생성
trip_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('learning_spark_data/fhvhv_tripdata_2020-03.csv')
zone_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('learning_spark_data/taxi+_zone_lookup.csv')

In [ ]:
# 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [46]:
trip_df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [47]:
zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [52]:
trip_df.createOrReplaceTempView('trip')
zone_df.createOrReplaceTempView('zone')

In [56]:
# 승차 Location(PULocationID)별 개수 세기
query = """
SELECT PULocationID,
Zone,
count(*)
FROM trip
INNER JOIN zone
ON trip.PULocationID = zone.LocationID
GROUP BY PULocationID,
Zone
"""
spark.sql(query).show()

+------------+--------------------+--------+
|PULocationID|                Zone|count(1)|
+------------+--------------------+--------+
|         181|          Park Slope|  121576|
|         182|         Parkchester|   39914|
|         166| Morningside Heights|   70728|
|         250|Westchester Villa...|   36232|
|         168|Mott Haven/Port M...|   98424|
|          18|        Bedford Park|   81206|
|          64|          Douglaston|    6841|
|          14|           Bay Ridge|   69667|
|          26|        Borough Park|   79057|
|         207|Saint Michaels Ce...|    2999|
|         243|Washington Height...|   87431|
|          16|             Bayside|   23875|
|         139|           Laurelton|   22500|
|         115| Grymes Hill/Clifton|   10806|
|         259|  Woodlawn/Wakefield|   42591|
|         232|Two Bridges/Sewar...|   64543|
|         162|        Midtown East|  104300|
|         201|       Rockaway Park|    6472|
|          29|      Brighton Beach|   27273|
|         

In [57]:
# 하차 Location(DOLocationID)별 개수 세기
query = """
SELECT DOLocationID,
Zone,
count(*)
FROM trip
INNER JOIN zone
ON trip.DOLocationID = zone.LocationID
GROUP BY DOLocationID,
Zone
"""
spark.sql(query).show()

+------------+--------------------+--------+
|DOLocationID|                Zone|count(1)|
+------------+--------------------+--------+
|         181|          Park Slope|  116900|
|         182|         Parkchester|   36309|
|         166| Morningside Heights|   68514|
|         250|Westchester Villa...|   37051|
|          18|        Bedford Park|   81271|
|         168|Mott Haven/Port M...|   99627|
|          14|           Bay Ridge|   71770|
|          26|        Borough Park|   81084|
|          64|          Douglaston|    6778|
|         207|Saint Michaels Ce...|    3467|
|         243|Washington Height...|   86795|
|          16|             Bayside|   23671|
|         139|           Laurelton|   18863|
|         115| Grymes Hill/Clifton|    9809|
|         259|  Woodlawn/Wakefield|   36600|
|         232|Two Bridges/Sewar...|   55726|
|         162|        Midtown East|   95681|
|         201|       Rockaway Park|    7269|
|          29|      Brighton Beach|   28095|
|         

In [58]:
#HV0003 운송사업자의 승차 지역별 트립 건수를 집계
query = '''
SELECT PULocationID,
Zone,
count(*)
FROM trip
INNER JOIN zone
ON trip.PULocationID = zone.LocationID
WHERE hvfhs_license_num = 'HV0003'
GROUP BY PULocationID,
Zone
'''
spark.sql(query).show()

+------------+--------------------+--------+
|PULocationID|                Zone|count(1)|
+------------+--------------------+--------+
|         181|          Park Slope|   79235|
|         182|         Parkchester|   33478|
|         166| Morningside Heights|   46653|
|         250|Westchester Villa...|   30788|
|          18|        Bedford Park|   70536|
|          64|          Douglaston|    5545|
|          14|           Bay Ridge|   55860|
|          26|        Borough Park|   67620|
|         168|Mott Haven/Port M...|   82396|
|         207|Saint Michaels Ce...|     664|
|         243|Washington Height...|   69058|
|          16|             Bayside|   19491|
|         139|           Laurelton|   17357|
|         115| Grymes Hill/Clifton|    8830|
|         259|  Woodlawn/Wakefield|   34254|
|         232|Two Bridges/Sewar...|   36390|
|         162|        Midtown East|   71730|
|         201|       Rockaway Park|    5356|
|          29|      Brighton Beach|   23249|
|         

In [63]:
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리
query = '''
SELECT hvfhs_license_num,
count(*)
FROM trip
GROUP BY hvfhs_license_num
ORDER BY count(*) DESC
'''
spark.sql(query).show()

+-----------------+--------+
|hvfhs_license_num|count(1)|
+-----------------+--------+
|           HV0003| 9836763|
|           HV0005| 3219535|
|           HV0004|  336606|
+-----------------+--------+



In [64]:
#승차 위치 Borough별 운행 건수
query = """
SELECT PULocationID,
Borough,
count(*)
FROM trip
INNER JOIN zone
ON trip.PULocationID = zone.LocationID
GROUP BY PULocationID,
Borough
"""
spark.sql(query).show()

+------------+-------------+--------+
|PULocationID|      Borough|count(1)|
+------------+-------------+--------+
|         215|       Queens|   44136|
|          32|        Bronx|   42128|
|         232|    Manhattan|   64543|
|          34|     Brooklyn|   11823|
|         235|        Bronx|   83347|
|         168|        Bronx|   98424|
|         260|       Queens|   52982|
|         118|Staten Island|   12375|
|          88|    Manhattan|   26235|
|          80|     Brooklyn|   98314|
|          50|    Manhattan|   80762|
|          92|       Queens|   54928|
|         216|       Queens|   77366|
|         122|       Queens|   16043|
|           9|       Queens|   10448|
|         117|       Queens|   18570|
|         186|    Manhattan|   79729|
|         107|    Manhattan|  102656|
|          62|     Brooklyn|   57126|
|         191|       Queens|   33139|
+------------+-------------+--------+
only showing top 20 rows



In [77]:
#서비스 존별 승차/하차 건수
query = """
SELECT service_zone,
PULocationID,
count(*) AS PUCount
FROM trip
INNER JOIN zone
ON trip.PULocationID = zone.LocationID
GROUP BY service_zone,
PULocationID
"""
spark.sql(query).show()

+------------+------------+-------+
|service_zone|PULocationID|PUCount|
+------------+------------+-------+
|   Boro Zone|         198|  82045|
|   Boro Zone|         136|  48398|
|   Boro Zone|         240|   8457|
|   Boro Zone|          89| 124289|
|   Boro Zone|          81|  41425|
|   Boro Zone|          31|   5285|
|   Boro Zone|          22|  46127|
|   Boro Zone|          11|  15737|
|   Boro Zone|          39| 116669|
|   Boro Zone|         241|  65961|
|   Boro Zone|         154|   1242|
|   Boro Zone|         147|  40262|
| Yellow Zone|         162| 104300|
|   Boro Zone|          84|   5235|
|   Boro Zone|         189|  45973|
|   Boro Zone|          78|  76155|
|   Boro Zone|          17| 126228|
|   Boro Zone|         196|  32282|
|   Boro Zone|         243|  87431|
|   Boro Zone|          15|   7685|
+------------+------------+-------+
only showing top 20 rows



In [78]:
query = """
SELECT service_zone,
DOLocationID,
count(*) AS DOCount
FROM trip
INNER JOIN zone
ON trip.PULocationID = zone.LocationID
GROUP BY service_zone,
DOLocationID
"""
spark.sql(query).show()

+------------+------------+-------+
|service_zone|DOLocationID|DOCount|
+------------+------------+-------+
|         EWR|         181|     11|
|   Boro Zone|         136|  46285|
|   Boro Zone|         198|  74176|
|    Airports|          81|    226|
|   Boro Zone|         240|   8234|
| Yellow Zone|         242|   1771|
|   Boro Zone|          89| 117246|
|   Boro Zone|         234|  22194|
| Yellow Zone|         115|    270|
|    Airports|          39|   1457|
|   Boro Zone|          81|  37303|
| Yellow Zone|         106|   2806|
| Yellow Zone|          91|   1918|
|    Airports|         191|    706|
|    Airports|         134|   2070|
|   Boro Zone|          31|   4992|
|    Airports|         232|   1033|
| Yellow Zone|          19|    351|
|    Airports|          93|    116|
|   Boro Zone|          22|  43660|
+------------+------------+-------+
only showing top 20 rows



In [79]:
query = """
SELECT
    service_zone,
    LocationID,
    SUM(CASE WHEN Type = 'PU' THEN Count ELSE 0 END) AS PUCount,
    SUM(CASE WHEN Type = 'DO' THEN Count ELSE 0 END) AS DOCount
FROM (
    SELECT
        z.service_zone,
        t.PULocationID AS LocationID,
        'PU' AS Type,
        count(*) AS Count
    FROM trip t
    JOIN zone z
    ON t.PULocationID = z.LocationID
    GROUP BY z.service_zone, t.PULocationID

    UNION ALL

    SELECT
        z.service_zone,
        t.DOLocationID AS LocationID,
        'DO' AS Type,
        count(*) AS Count
    FROM trip t
    JOIN zone z
    ON t.DOLocationID = z.LocationID
    GROUP BY z.service_zone, t.DOLocationID
) AS subquery
GROUP BY service_zone, LocationID
ORDER BY service_zone, LocationID
"""

spark.sql(query).show()

+------------+----------+-------+-------+
|service_zone|LocationID|PUCount|DOCount|
+------------+----------+-------+-------+
|    Airports|       132| 163734| 216213|
|    Airports|       138| 155876| 194943|
|   Boro Zone|         2|     23|     21|
|   Boro Zone|         3|  34076|  32728|
|   Boro Zone|         5|   3189|   3126|
|   Boro Zone|         6|   6283|   6541|
|   Boro Zone|         7| 125458| 124255|
|   Boro Zone|         8|    297|    340|
|   Boro Zone|         9|  10448|  10554|
|   Boro Zone|        10|  44290|  36941|
|   Boro Zone|        11|  15737|  15403|
|   Boro Zone|        14|  69667|  71770|
|   Boro Zone|        15|   7685|   8059|
|   Boro Zone|        16|  23875|  23671|
|   Boro Zone|        17| 126228| 128152|
|   Boro Zone|        18|  81206|  81271|
|   Boro Zone|        19|   8672|   7633|
|   Boro Zone|        20|  48808|  46187|
|   Boro Zone|        21|  37106|  36122|
|   Boro Zone|        22|  46127|  46493|
+------------+----------+-------+-

In [82]:
from pyspark.sql.functions import col, count, lit, sum, when
# 1. 픽업(PU) 데이터프레임 생성
# trip_df를 zone_df와 PULocationID 기준으로 조인
# service_zone, PULocationID별로 count를 계산하고 Type을 'PU'로 지정
pu_df = trip_df.join(zone_df, trip_df.PULocationID == zone_df.LocationID) \
               .groupBy(zone_df.service_zone, trip_df.PULocationID.alias("LocationID")) \
               .agg(count("*").alias("Count")) \
               .withColumn("Type", lit("PU"))

# 2. 하차(DO) 데이터프레임 생성
# trip_df를 zone_df와 DOLocationID 기준으로 조인
# service_zone, DOLocationID별로 count를 계산하고 Type을 'DO'로 지정
do_df = trip_df.join(zone_df, trip_df.DOLocationID == zone_df.LocationID) \
               .groupBy(zone_df.service_zone, trip_df.DOLocationID.alias("LocationID")) \
               .agg(count("*").alias("Count")) \
               .withColumn("Type", lit("DO"))

# 3. PU와 DO 데이터프레임 통합 (UNION ALL)
# pu_df와 do_df를 합칩니다.
combined_df = pu_df.unionAll(do_df)

# 4. 최종 집계
# 통합된 데이터프레임에서 service_zone과 LocationID로 그룹화
# CASE WHEN 로직을 `sum`과 `when` 함수로 구현하여 PUCount와 DOCount를 계산
final_df = combined_df.groupBy("service_zone", "LocationID") \
                      .agg(sum(when(col("Type") == "PU", col("Count")).otherwise(0)).alias("PUCount"),
                           sum(when(col("Type") == "DO", col("Count")).otherwise(0)).alias("DOCount")) \
                      .orderBy("service_zone", "LocationID")

# 결과 출력
final_df.show()

+------------+----------+-------+-------+
|service_zone|LocationID|PUCount|DOCount|
+------------+----------+-------+-------+
|    Airports|       132| 163734| 216213|
|    Airports|       138| 155876| 194943|
|   Boro Zone|         2|     23|     21|
|   Boro Zone|         3|  34076|  32728|
|   Boro Zone|         5|   3189|   3126|
|   Boro Zone|         6|   6283|   6541|
|   Boro Zone|         7| 125458| 124255|
|   Boro Zone|         8|    297|    340|
|   Boro Zone|         9|  10448|  10554|
|   Boro Zone|        10|  44290|  36941|
|   Boro Zone|        11|  15737|  15403|
|   Boro Zone|        14|  69667|  71770|
|   Boro Zone|        15|   7685|   8059|
|   Boro Zone|        16|  23875|  23671|
|   Boro Zone|        17| 126228| 128152|
|   Boro Zone|        18|  81206|  81271|
|   Boro Zone|        19|   8672|   7633|
|   Boro Zone|        20|  48808|  46187|
|   Boro Zone|        21|  37106|  36122|
|   Boro Zone|        22|  46127|  46493|
+------------+----------+-------+-

In [95]:
#각 탑승의 총 소요 시간을 분 단위로 계산하고, 이 시간이 30분 이상인 탑승 건수를 세세요.
query = '''
SELECT COUNT(DATEDIFF(minute,pickup_datetime,dropoff_datetime)) as boardingTimeOver30Minute
FROM trip
WHERE DATEDIFF(minute,pickup_datetime,dropoff_datetime) >= 30
'''
spark.sql(query).show()

+------------------------+
|boardingTimeOver30Minute|
+------------------------+
|                 1530626|
+------------------------+

